In [1]:
import os, requests
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

search_en_id = os.getenv('search_en_id')
api_key = os.getenv('api_key')

In [112]:
import google.generativeai as genai
from langchain.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from duckduckgo_search import DDGS

from langchain_google_genai import (ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings,HarmBlockThreshold,HarmCategory)

from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS

from langchain_community.document_loaders import WebBaseLoader

import nest_asyncio



In [87]:
safe ={
	HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
	HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
	}

generation_config = {
  "temperature": 0,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 10000,
}

In [4]:
for m in genai.list_models():
	if 'generateContent' in m.supported_generation_methods:
		print(m.name)

model = "gemini-1.0-pro-vision-latest"

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [6]:
url_ = "https://www.googleapis.com/customsearch/v1"

def build_params(search_query, num = 10, start=1, dateRestrict='d1',**kwargs):
	params = { 'q': search_query, 
		   	   'key': api_key, 
			   'cx': search_en_id,
			   'num': num,
			   'start': 1,
			   'dateRestrict': 'd1',
			   }
	params.update(kwargs)
	return params

def google_search(params):
	response = requests.get(url_, params=params)
	if response.status_code != 200:
		raise Exception('API response: {}'.format(response.status_code))
	return response.json()

In [7]:
# search = DuckDuckGoSearchResults()
# res = search.run("Obamna", max_results=2)
# wrapper = DuckDuckGoSearchAPIWrapper( max_results=2)
# search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news")
# res = search.run("Obamna")

# Searching for pdf files
# results = DDGS().text('russia filetype:pdf', region='us-en', safesearch='off', timelimit='y', max_results=10)

# async


In [114]:
query = "anime and 'surface tension'"
results = DDGS().text(query, region='us-en', safesearch='off', timelimit='y', max_results=20)
results

[{'title': 'Mastering Drawing Anime Hands: A Step-by-Step Guide - Daisie Blog',
  'href': 'https://blog.daisie.com/mastering-drawing-anime-hands-a-step-by-step-guide/',
  'body': "With the right tools and techniques, you can master the art of drawing anime hands. But before you put your pencil down, there's one more step—polishing and finalizing your drawing. 5. Polishing and Finalizing Your Drawing. Now comes the final step in our drawing anime hands guide—polishing and finalizing your drawing."},
 {'title': 'Surface tension | Definition, Examples, & Facts | Britannica',
  'href': 'https://www.britannica.com/science/surface-tension',
  'body': 'Surface tension depends mainly upon the forces of attraction between the particles within the given liquid and also upon the gas, solid, or liquid in contact with it.The molecules in a drop of water, for example, attract each other weakly. Water molecules well inside the drop may be thought of as being attracted equally in all directions by the

In [8]:
query = "obamna"

In [9]:

results = DDGS().text('Obamna', region='us-en', safesearch='off', timelimit='y', max_results=20)


In [10]:
results[:2]

[{'title': "Where Did The Nickname 'Obamna' Come From? Memes About Trump's ...",
  'href': 'https://knowyourmeme.com/editorials/guides/where-did-the-nickname-obamna-come-from-memes-about-trumps-mispronounciation-of-obamas-name-explained',
  'body': 'Obamna is a nickname for former President Barack Obama that originated from Trump\'s apparent mix-up of his name with amnesty in 2018. The word has been used in many memes, especially with Biden\'s "soda" clip, to mock Trump\'s gaffes and humor.'},
 {'title': 'Barack Obama: Biography, 44th U.S. President, Politician',
  'href': 'https://www.biography.com/political-figures/barack-obama',
  'body': 'Learn about the life and achievements of Barack Obama, the first Black commander-in-chief of the United States. Find out his early years, family background, education, political career, awards, and legacy.'}]

In [11]:
meta = [{"href": f"{r['href']}", "title": f"{r['title']}"} for r in results]
txt = [r['body'] for r in results]

In [12]:
meta[:2]

[{'href': 'https://knowyourmeme.com/editorials/guides/where-did-the-nickname-obamna-come-from-memes-about-trumps-mispronounciation-of-obamas-name-explained',
  'title': "Where Did The Nickname 'Obamna' Come From? Memes About Trump's ..."},
 {'href': 'https://www.biography.com/political-figures/barack-obama',
  'title': 'Barack Obama: Biography, 44th U.S. President, Politician'}]

In [13]:
# """Enumerator of the Distance strategies for calculating distances
# between vectors."""

# EUCLIDEAN_DISTANCE = "EUCLIDEAN_DISTANCE"
# MAX_INNER_PRODUCT = "MAX_INNER_PRODUCT"
# DOT_PRODUCT = "DOT_PRODUCT"
# JACCARD = "JACCARD"
# COSINE = "COSINE" 

In [14]:
distance_strategy = "EUCLIDEAN_DISTANCE"
db_p = FAISS.from_texts(txt, embeddings, metadatas=meta, distance_strategy=distance_strategy)
# db_p.distance_strategy = "DOT_PRODUCT"

In [15]:
db_p.distance_strategy

'EUCLIDEAN_DISTANCE'

In [16]:
docs_and_scores = db_p.similarity_search_with_score(query, k=10)

In [17]:
docs_and_scores[::-1][:5]

[(Document(page_content="Biden, 81, is the oldest president in history and Trump, 77, would be the second-oldest if he wins in November. And they're increasingly frequent misspoken comments have drawn increased scrutiny ...", metadata={'href': 'https://www.forbes.com/sites/saradorn/2024/03/17/biden-trump-gaffe-tracker-trump-incorrectly-brings-up-obama-again/', 'title': 'Biden-Trump Gaffe Tracker: Trump Incorrectly Brings Up Obama ... - Forbes'}),
  0.52686834),
 (Document(page_content='Mr. Obama looked at him evenly. "Well," he replied, "Senator Kerry is not president of the United States.". That moment of sharp relief, a clash with an intransigent foreign apparatchik by ...', metadata={'href': 'https://www.nytimes.com/2023/05/31/us/politics/obama-oral-history.html', 'title': 'What a New Oral History Reveals About Obama, and the Tradeoffs He Made'}),
  0.49196023),
 (Document(page_content="Michelle Obama (born January 17, 1964, Chicago, Illinois, U.S.) American first lady (2009-17), th

In [18]:
links = [d[0].metadata['href'] for d in docs_and_scores[::-1]]
meta = [{"href": f"{r['href']}", "title": f"{r['title']}"} for r in results]

meta_ = [{"link":d[0].metadata['href'], "title":d[0].metadata['title'], "score":d[1]} for d in docs_and_scores[::-1]]
links

['https://www.forbes.com/sites/saradorn/2024/03/17/biden-trump-gaffe-tracker-trump-incorrectly-brings-up-obama-again/',
 'https://www.nytimes.com/2023/05/31/us/politics/obama-oral-history.html',
 'https://www.britannica.com/biography/Michelle-Obama',
 'https://edition.cnn.com/2023/12/15/politics/biden-obama-affordable-care-act/index.html',
 'https://knowyourmeme.com/editorials/guides/where-did-the-nickname-obamna-come-from-memes-about-trumps-mispronounciation-of-obamas-name-explained',
 'https://www.cnn.com/2024/01/20/politics/obama-inauguration-america-blake-cec/index.html',
 'https://www.businessinsider.com/biden-obama-staffers-presidential-administration-effectiveness-2024-3?op=1',
 'https://www.youtube.com/watch?v=NwLOuwtPAJ4',
 'https://www.britannica.com/biography/Barack-Obama',
 'https://www.theguardian.com/us-news/2024/mar/18/barack-obama-rishi-sunak-downing-street-london-visit']

### Google

In [115]:
response = google_search(build_params(query,num = 10))

In [116]:
for i in response['items']:
	url, title = i['link'], i['title']
	print(f"URL : {url}  title : {title}")
	print(f"Snippet : {i['snippet']}")

URL : https://www.reddit.com/r/nosleep/  title : nosleep
Snippet : 4 hours ago ... Above us, dim pendant lights hang low, illuminating the polished surface of the counter. The walls were adorned with various vintage poster. Wooden tables and ...
URL : https://www.imdb.com/title/tt0214341/news/  title : Dragon Ball Z (TV Series 1989–1996) - News - IMDb
Snippet : 4 hours ago ... There needs to be a push and pull of tension, the ... The fans were overjoyed when they saw a Fortnite glitch surface online, with the players being able to play ...
URL : https://archiveofourown.org/works/54536896/chapters/138216652  title : The Purchased Alpha - Chapter 3 - Stephy_E_C - Death Note ...
Snippet : 18 hours ago ... Anime & Manga · Books & Literature · Cartoons ... Death Note (Anime & Manga) · The Purchased Alpha(Alpha ... tension that still hung heavy, despite the surface calm ...
URL : https://en.wikipedia.org/wiki/Jan_%C5%A0vankmajer  title : Jan Švankmajer - Wikipedia
Snippet : 5 hours ago ... T

In [21]:
from typing import Any, Dict, Iterator, List, Optional, Sequence, Union
from bs4 import BeautifulSoup
import re

def extract_text(html):
	soup = BeautifulSoup(html.content, 'html.parser')
	text = re.sub(r'\s+', ' ', soup.get_text()).strip()
	return text

class WebLoader(WebBaseLoader):
    # self.requests_kwargs.update({"headers": headers})
    def _scrape(self, url: str, parser: Union[str, None] = None, bs_kwargs: Optional[dict] = None,) -> Any:
        html_doc = self.session.get(url, **self.requests_kwargs)
        if self.raise_for_status:
            html_doc.raise_for_status()
        
        html_doc.encoding = html_doc.apparent_encoding
        text = extract_text(html_doc)
        return BeautifulSoup(text, 'html.parser')

In [22]:
nest_asyncio.apply()

In [23]:
%%time
headers = {'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"}

loader = WebLoader(links)
loader.session.headers = headers
loader.requests_per_second = 5
docs = loader.load()
docs

CPU times: user 18.6 s, sys: 175 ms, total: 18.7 s
Wall time: 24.4 s


[Document(page_content='Biden-Trump Gaffe Tracker: Trump Incorrectly Brings Up Obama—AgainSubscribe To NewslettersSign InBETAThis is a BETA experience. You may opt-out by clicking hereBreaking53 minutes agoWho Is Nicole Shanahan? RFK Jr.’s Possible Running Mate Is A Tech Lawyer Once Married To Google Founder Brin4 hours agoNetanyahu Blasts Schumer’s Call For Israeli Elections As ‘Totally Inappropriate’6 hours agoAn Even Darker Trump: Warns Of ‘Bloodbath’ If Not Elected And Calls Some Migrants ‘Animals’ And ‘Not People’6 hours agoClaims About BBC Being On-Alert For Kate Middleton News Remain Unconfirmed24 hours agoWho Is Vaughan Gething? What To Know About Europe’s First Black Government Leader.+1 day agoElon Musk’s SpaceX Developing Spy Satellite Network For The U.S., Report Says+1 day agoThree Killed In Philadelphia Suburb Shooting Spree, Police Say Gunman Is Barricaded In New Jersey Residence+1 day agoCease-Fire Talks Between Israel And Hamas Will Resume As Netanyahu Approves Rafah I

In [25]:
load_dotenv()
NEO4J_URI = "neo4j+s://ad479dea.databases.neo4j.io"
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


In [26]:
from langchain_community.vectorstores import Neo4jVector

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker

In [50]:
docs

[Document(page_content='Biden-Trump Gaffe Tracker: Trump Incorrectly Brings Up Obama—AgainSubscribe To NewslettersSign InBETAThis is a BETA experience. You may opt-out by clicking hereBreaking53 minutes agoWho Is Nicole Shanahan? RFK Jr.’s Possible Running Mate Is A Tech Lawyer Once Married To Google Founder Brin4 hours agoNetanyahu Blasts Schumer’s Call For Israeli Elections As ‘Totally Inappropriate’6 hours agoAn Even Darker Trump: Warns Of ‘Bloodbath’ If Not Elected And Calls Some Migrants ‘Animals’ And ‘Not People’6 hours agoClaims About BBC Being On-Alert For Kate Middleton News Remain Unconfirmed24 hours agoWho Is Vaughan Gething? What To Know About Europe’s First Black Government Leader.+1 day agoElon Musk’s SpaceX Developing Spy Satellite Network For The U.S., Report Says+1 day agoThree Killed In Philadelphia Suburb Shooting Spree, Police Say Gunman Is Barricaded In New Jersey Residence+1 day agoCease-Fire Talks Between Israel And Hamas Will Resume As Netanyahu Approves Rafah I

In [36]:
%%time
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=20)
text_splitter = SemanticChunker(embeddings, breakpoint_threshold_type="standard_deviation")
docs_s = text_splitter.split_documents(docs)

CPU times: user 1.6 s, sys: 43.1 ms, total: 1.64 s
Wall time: 6.65 s


In [52]:
db = Neo4jVector.from_documents(docs, embeddings, url = NEO4J_URI,search_type="hybrid")

In [55]:
docs_with_score = db.similarity_search_with_score(query, k=5)

In [56]:
docs_with_score

[(Document(page_content="Inside Obama's Presidency (full documentary) | FRONTLINE - YouTubeAboutPressCopyrightContact usCreatorsAdvertiseDevelopersTermsPrivacyPolicy & SafetyHow YouTube worksTest new featuresNFL Sunday Ticket© 2024 Google LLC", metadata={'source': 'https://www.youtube.com/watch?v=NwLOuwtPAJ4'}),
  1.0),
 (Document(page_content='Where Did The Nickname \'Obamna\' Come From? Memes About Trump\'s Mispronounciation Of Obama\'s Name Explained | Know Your Meme Advanced Search Protips About Rules Chat Random Activity Welcome! Login or signup now! Home Memes Confirmed Submissions Researching Newsworthy Popular Deadpool All Submit an Entry Categories Cultures Events Memes People Sites Subcultures Trending News Images Trending Most Commented Most Favorited Most Liked Least Liked Most Viewed All Templates Upload an Image Videos Trending Most Commented Most Favorited Most Liked Most Viewed All Upload a Video Forums Discussion General Meme Research Serious Debate Q & A Media Video G

In [57]:
retriever = db.as_retriever()
retriever.get_relevant_documents(query)[0]

Document(page_content="Inside Obama's Presidency (full documentary) | FRONTLINE - YouTubeAboutPressCopyrightContact usCreatorsAdvertiseDevelopersTermsPrivacyPolicy & SafetyHow YouTube worksTest new featuresNFL Sunday Ticket© 2024 Google LLC", metadata={'source': 'https://www.youtube.com/watch?v=NwLOuwtPAJ4'})

In [60]:
db_ = FAISS.from_documents(docs, embeddings, metadatas=meta_)

TypeError: langchain_community.vectorstores.faiss.FAISS.from_texts() got multiple values for keyword argument 'metadatas'

In [62]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_groq import ChatGroq

In [63]:
groq_api_key = os.environ['groq_api']

In [88]:
llm_g = ChatGoogleGenerativeAI(model="gemini-1.0-pro-latest", generation_config=generation_config,safety_settings=safe, convert_system_message_to_human=True)
llm_groq = ChatGroq(groq_api_key=groq_api_key,
            #model_name='llama2-70b-4096' 
            model_name='mixtral-8x7b-32768')

In [67]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [76]:
from langchain.retrievers.document_compressors import EmbeddingsFilter



embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.5)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)

Document 1:

Inside Obama's Presidency (full documentary) | FRONTLINE - YouTubeAboutPressCopyrightContact usCreatorsAdvertiseDevelopersTermsPrivacyPolicy & SafetyHow YouTube worksTest new featuresNFL Sunday Ticket© 2024 Google LLC
----------------------------------------------------------------------------------------------------
Document 2:

Obama shared his vision of America at his first inauguration. Some still believe | CNN Politics CNN values your feedback 1. How relevant is this ad to you? 2. Did you encounter any technical issues? Video player was slow to load content Video content never loaded Ad froze or did not finish loading Video content did not start after ad Audio on ad was too loud Other issues Ad never loaded Ad prevented/slowed the page from loading Content moved around while ad loaded Ad was repetitive to ads I've seen previously Other issues Cancel Submit Thank You! Your effort and contribution in providing this feedback is much appreciated. Close Ad Feedback Close i

In [81]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


compressor = LLMChainExtractor.from_llm(llm_g)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(query)

pretty_print_docs(compressed_docs)

/home/jrdfm/.local/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/jrdfm/.local/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/jrdfm/.local/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/jrdfm/.local/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

Where Did The Nickname 'Obamna' Come From? Memes About Trump's Mispronounciation Of Obama's Name Explained


During the rally, which took place in Houston, Texas, Trump seemingly mispronounced Obama's name as "Obamna," perhaps mixing it up with the word "amnesty". The clarity with which Trump delivered and indeed paused on the word "Obamna" was the cause of much hilarity for onlookers regardless of their political views.
----------------------------------------------------------------------------------------------------
Document 2:

"The Obama people thought Biden would suck as president," an ex-Biden aide recently told Axios.


In [83]:
compressed_docs

[Document(page_content='Where Did The Nickname \'Obamna\' Come From? Memes About Trump\'s Mispronounciation Of Obama\'s Name Explained\n\n\nDuring the rally, which took place in Houston, Texas, Trump seemingly mispronounced Obama\'s name as "Obamna," perhaps mixing it up with the word "amnesty". The clarity with which Trump delivered and indeed paused on the word "Obamna" was the cause of much hilarity for onlookers regardless of their political views.', metadata={'source': 'https://knowyourmeme.com/editorials/guides/where-did-the-nickname-obamna-come-from-memes-about-trumps-mispronounciation-of-obamas-name-explained'}),
 Document(page_content='"The Obama people thought Biden would suck as president," an ex-Biden aide recently told Axios.', metadata={'source': 'https://www.businessinsider.com/biden-obama-staffers-presidential-administration-effectiveness-2024-3?op=1'})]

In [95]:
from langchain.chains import RetrievalQA
from IPython.display import Markdown, display

In [96]:
def printmd(string):
	display(Markdown(string))

In [91]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
qa = RetrievalQA.from_chain_type(
    llm=llm_groq, chain_type="stuff", retriever=retriever, return_source_documents=True)
query = "Who is obmana"
result = qa({"query": query})

In [97]:
printmd(result['result'])

"Obamna" is a nickname or misspelling of the name "Obama," which originated from a verbal slip by former President Donald Trump during a rally in Houston, Texas, in October 2018. Trump appeared to mispronounce the name of then-former President Barack Obama as "Obamna," possibly mixing it up with the word "amnesty." The clip of Trump saying "Obamna" has since been used in memes and combined with other presidential memes, such as a clip of President Joe Biden appearing to say "soda." The moniker is not associated with any particular person or character named "Obamna."

In [101]:
# ['stuff', 'map_reduce', 'refine', 'map_rerank']

In [104]:
%%time
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 15})
qa = RetrievalQA.from_chain_type(
    llm=llm_g, chain_type="refine", retriever=retriever, return_source_documents=True)
query = "Who is obmana"
result = qa({"query": query})
printmd(result['result'])

**Refined Answer:**

Barack Obama is the 44th President of the United States. He served two terms in office from 2009 to 2017. During his presidency, he oversaw the passage of the Affordable Care Act, also known as Obamacare.

In recent news, Obama visited London and met with Prime Minister Rishi Sunak. The two discussed a range of topics, including artificial intelligence. Obama also met with Labour leader Keir Starmer.

The nickname "Obamna" was given to Obama by Donald Trump. The nickname originated from a 2018 rally where Trump mispronounced Obama's name as "Obamna." The clip of Trump saying "Obamna" has since become a popular meme.

**Additionally, it's worth noting that the nickname "Obamna" has been used by some to mock or belittle former President Obama. This is particularly true among Trump supporters, who often use the nickname as a way to express their dislike or disagreement with Obama's policies or actions.**

**In the context of the provided text, it is important to note that Michelle Obama, the former First Lady, played a significant role in her husband's political career. She was an active campaigner during his presidential runs and was known for her speeches and public appearances. Michelle Obama also focused on various causes as First Lady, including supporting military families and ending childhood obesity.**

CPU times: user 222 ms, sys: 56.6 ms, total: 278 ms
Wall time: 52.8 s


In [111]:
%%time
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})
qa = RetrievalQA.from_chain_type(
    llm=llm_groq, chain_type="refine", retriever=retriever, return_source_documents=True)
query = "Who is obmana"
result = qa({"query": query})
printmd(result['result'])

The provided context is a biography of Michelle Obama, the wife of Barack Obama, the 44th President of the United States. The biography includes information about Michelle's background, education, career, and her role as the First Lady during her husband's presidency. The biography does not directly relate to the term "Obamna" but provides additional context about the Obama family.

The original answer remains valid:

Obaman(a) is a nickname or variation of the name "Obama" that originated from a misspoken word by former President Donald Trump. During a rally in Houston, Texas, in October 2018, Trump seemingly mispronounced then-President Barack Obama's name as "Obamna" while discussing the topic of amnesty. The mispronunciation was unintentional and became a source of humor and meme fodder on the internet. The name "Obamna" has been used in memes and online discussions, often in conjunction with other presidential memes or political humor.

CPU times: user 280 ms, sys: 30.1 ms, total: 311 ms
Wall time: 14.5 s


In [109]:
%%time
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 15})
qa = RetrievalQA.from_chain_type(
    llm=llm_g, chain_type="map_rerank", retriever=retriever, return_source_documents=True)
query = "Who is obmana"
result = qa({"query": query})
printmd(result['result'])

/home/jrdfm/.local/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Obamna is a nickname for former President Barack Obama. The nickname originated from a 2018 rally when President Trump mispronounced Obama's name as "Obamna," perhaps mixing it up with the word "amnesty". The clarity with which Trump delivered and indeed paused on the word "Obamna" was the cause of much hilarity for onlookers regardless of their political views.

CPU times: user 150 ms, sys: 34.3 ms, total: 184 ms
Wall time: 33.2 s


In [117]:
results = DDGS().suggestions(query)

# async
results 

[{'phrase': 'fight tension school anime'}]

### Suggestions/ related

In [118]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')
response = model.generate_content("Generate a more detailed internet search query based on this key word: " + query, safety_settings=safe)

print(response.text)

* Anime featuring characters using surface tension powers
* Surface tension as a plot device in anime
* Anime with water manipulation abilities based on surface tension
* Scientific principles of surface tension in anime
* Visual representations of surface tension in anime animation
